In [1]:
from everywhereml.data import Dataset
from everywhereml.data.collect import SerialCollector
from everywhereml.preprocessing import Pipeline, MinMaxScaler, Window, SpectralFeatures
from everywhereml.sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c:\Users\Hersh\Anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Add the port your Arduino is connected to

Collect Training Data

In [79]:
# port = '/dev/cu.usbmodem141401'
port = 'COM7'

try:
    imu_dataset = Dataset.from_csv(
        'imu.csv', 
        name='ContinuousMotion', 
        target_name_column='target_name'
    )
    
except FileNotFoundError:
    imu_collector = SerialCollector(
        port=port, 
        baud=115200, 
        start_of_frame='IMU:', 
        feature_names=['ax', 'ay', 'az', 'gx', 'gy', 'gz']
    )
    imu_dataset = imu_collector.collect_many_classes(
        dataset_name='ContinuousMotion', 
        duration=30
    )
    
    # save dataset to file for later use
    imu_dataset.df.to_csv('imu.csv', index=False)

print(imu_dataset.df.columns)
print(imu_dataset.df['target_name'].value_counts())

Index(['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'target', 'target_name'], dtype='object')
target_name
vertical_shake      3308
neutral             3275
punch               3268
clockwise           3240
counterclockwise    3222
horizontal_shake    3217
Name: count, dtype: int64


## Classes to Use

In [84]:
# This dictionary is used to enable/disable certain gestures
# comment out unwanted gestures
target_names_to_use = [
    "vertical_shake",
    "horizontal_shake",
    "neutral",
    # "punch",
    "clockwise",
    "counterclockwise"
]

# Filter out gestures that are not enabled
imu_dataset.df = imu_dataset.df[imu_dataset.df['target_name'].isin(target_names_to_use)]
for i, target_name in enumerate(target_names_to_use):
    imu_dataset.df.loc[imu_dataset.df['target_name'] == target_name, "target"]["target"] = i



In [85]:
# this is the frequency of your sensor
# change according to your hardware
sampling_frequency = 104
mean_gesture_duration_in_millis = 1000
window_length = sampling_frequency * mean_gesture_duration_in_millis // 1000

imu_pipeline = Pipeline(name='ContinousMotionPipeline', steps=[
    MinMaxScaler(),
    # shift can be an integer (number of samples) or a float (percent)
    Window(length=window_length, shift=0.3),
    # order can either be 1 (first-order features) or 2 (add second-order features)
    SpectralFeatures(order=2)
])

pprint(imu_pipeline['SpectralFeatures'][0].feature_names)

"""
Apply feature pre-processing
"""
imu_dataset.apply(imu_pipeline)
imu_dataset.describe()

['maximum',
 'minimum',
 'abs_maximum',
 'abs_minimum',
 'mean',
 'abs_energy',
 'mean_abs_change',
 'cid_ce',
 'std',
 'var',
 'count_above_mean',
 'count_below_mean',
 'first_position_of_max',
 'first_position_of_min',
 'max_count',
 'min_count',
 'has_large_std',
 'skew',
 'kurtosis',
 'variation_coefficient']


c:\Users\Hersh\Anaconda3\envs\py311\Lib\site-packages\everywhereml\preprocessing\SpectralFeatures.py:150: RuntimeWarning: invalid value encountered in divide
  skew = np.where(var < eps, 0, ((series - mean) ** 3) / (var ** 1.5)).mean(axis=1)
c:\Users\Hersh\Anaconda3\envs\py311\Lib\site-packages\everywhereml\preprocessing\SpectralFeatures.py:151: RuntimeWarning: invalid value encountered in divide
  kurtosis = np.where(np.abs(var) < eps, 0, ((series - mean) ** 4) / (var ** 2)).mean(axis=1)
c:\Users\Hersh\Anaconda3\envs\py311\Lib\site-packages\everywhereml\preprocessing\SpectralFeatures.py:152: RuntimeWarning: invalid value encountered in divide
  variation_coefficient = np.where(mean < eps, 0, std / mean)


,ax_maximum_maximum,ax_maximum_minimum,ax_maximum_abs_maximum,ax_maximum_abs_minimum,ax_maximum_mean,ax_maximum_abs_energy,ax_maximum_mean_abs_change,ax_maximum_cid_ce,ax_maximum_std,ax_maximum_var,...,gz_variation_coefficient_count_below_mean,gz_variation_coefficient_first_position_of_max,gz_variation_coefficient_first_position_of_min,gz_variation_coefficient_max_count,gz_variation_coefficient_min_count,gz_variation_coefficient_has_large_std,gz_variation_coefficient_skew,gz_variation_coefficient_kurtosis,gz_variation_coefficient_variation_coefficient,target
count,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,...,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
mean,0.423667,0.073831,0.423667,0.073831,0.201427,0.067441,0.028213,0.008161,0.088014,0.013210,...,56.352941,49.941176,40.000000,1.235294,0.941176,0.529412,0.444188,2.566404,0.634221,2.352941
std,0.314510,0.065502,0.314510,0.065502,0.120460,0.076365,0.035202,0.014879,0.076194,0.020320,...,14.075427,32.751852,31.398646,0.752447,0.555719,0.514496,0.834716,2.022855,0.326152,1.538716
min,0.131965,0.000000,0.131965,0.000000,0.069479,0.007094,0.002335,0.000037,0.013680,0.000187,...,29.000000,1.000000,1.000000,1.000000,0.000000,0.000000,-0.919859,0.000000,0.310021,0.000000
25%,0.173021,0.014663,0.173021,0.014663,0.104952,0.015309,0.006548,0.000251,0.041112,0.001690,...,51.000000,24.000000,14.000000,1.000000,1.000000,0.000000,0.000000,1.696064,0.418500,1.000000
50%,0.287390,0.079179,0.287390,0.079179,0.162164,0.029846,0.011161,0.000389,0.052888,0.002797,...,56.000000,55.000000,37.000000,1.000000,1.000000,1.000000,0.182497,2.108123,0.499757,2.000000
75%,0.563050,0.105572,0.563050,0.105572,0.262210,0.090908,0.029952,0.004190,0.122299,0.014957,...,63.000000,86.000000,56.000000,1.000000,1.000000,1.000000,0.566991,2.879337,0.817453,4.000000
max,1.000000,0.187683,1.000000,0.187683,0.457252,0.241627,0.112462,0.042595,0.266047,0.070781,...,90.000000,96.000000,102.000000,4.000000,2.000000,1.000000,2.139059,7.049332,1.298465,5.000000


In [86]:
"""
Plot features pairplot after feature extraction
Now it will start to make sense
Since SpectralFeatures generates 8 or 20 features (depending on the order)
for each axis, we limit the visualization to a more reasonable number
"""
# imu_dataset.plot.features_pairplot(n=300, k=6)

'\nPlot features pairplot after feature extraction\nNow it will start to make sense\nSince SpectralFeatures generates 8 or 20 features (depending on the order)\nfor each axis, we limit the visualization to a more reasonable number\n'

In [87]:
"""
Perform classification with a RandomForest
"""
imu_classifier = RandomForestClassifier(n_estimators=20, max_depth=20)
imu_train, imu_test = imu_dataset.split(test_size=0.3)
imu_classifier.fit(imu_train)

print('Score on test set: %.2f' % imu_classifier.score(imu_test))


# Plot confusion matrix
# If any off-diagonal values are dark, it means the gestures are too similar
# and the classifier is confusing them
df = imu_test.df
y_true = imu_test.df['target'].astype(int)
y_pred = imu_classifier.predict(imu_test.df.drop(columns=['target_name', 'target']))

mat = confusion_matrix(y_true, y_pred)
target_map = {i: name for i, name in enumerate(imu_test.target_names)}
plt.xticks(list(target_map.keys()), list(target_map.values()), rotation=90)
plt.yticks(list(target_map.keys()), list(target_map.values()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.imshow(mat, cmap='Blues', interpolation='nearest')
plt.show()
# round printed values to 2 decimal places
np.set_printoptions(precision=2)
print(mat / mat.max())

AssertionError: target_names MUST be None or have the same length as the number of labels

In [ ]:
"""
Port pipeline to C++
"""
print(imu_pipeline.to_arduino_file(
    '../Arduino/IMUClassify/Pipeline.h', 
    instance_name='pipeline'
))

# Pipeline.h file has a typo. Correct this
replacement_lines = [
    "\t\t\t\t\tstep0.transform(X)\n",
    "\t\t\t\t\t\n",
    "\t\t\t\t\t&& step1.transform(X)\n",
    "\t\t\t\t\t\n",
    "\t\t\t\t\t&& step2.transform(X)\n",
]

pipeline_file = '../Arduino/IMUClassify/Pipeline.h'

with open(pipeline_file, 'r') as f:
    file_data = f.readlines()

    with open(pipeline_file, 'w') as f:
        for i, line in enumerate(file_data):
            if i >= 262 and i < 267:
                line = replacement_lines[i-262]
            f.write(line)    


"""
Port classifier to C++
"""
print(imu_classifier.to_arduino_file(
    '../Arduino/IMUClassify/Classifier.h', 
    instance_name='forest', 
    class_map=imu_dataset.class_map
))

#ifndef UUID1764132504400
#define UUID1764132504400

#include <cstring>


namespace ContinousMotionPipeline {

    
        #ifndef UUID1764038226448
#define UUID1764038226448

/**
  * MinMaxScaler(low=0, high=1)
 */
class Step0 {
    public:

        /**
         * Transform input vector
         */
        bool transform(float *x) {
            
    for (uint16_t i = 0; i < 6; i++) {
        x[i] = (x[i] - offset[i]) * scale[i] + 0;

        if (x[i] < 0) x[i] = 0;
        else if (x[i] > 1) x[i] = 1;
    }

    return true;


            return true;
        }

    protected:
        
    float offset[6] = {-2.92000000000f, -2.25000000000f, -2.04000000000f, -1058.96000000000f, -209.11000000000f, -245.97000000000f};
    float scale[6] = {0.16583747927f, 0.17006802721f, 0.20449897751f, 0.00043942523f, 0.00212282676f, 0.00195304871f};

};



#endif
    
        #ifndef UUID1764110587280
#define UUID1764110587280

/**
  * Window(length=104, shift=31)
 */
class Step1 {
    public:

     